# Run hacked AlphaFold2 on the MPNN designed binders

### Imports

In [1]:
%load_ext lab_black
# Python standard library
from glob import glob
import os
import socket
import sys

# 3rd party library imports
import dask
import matplotlib.pyplot as plt
import pandas as pd
import pyrosetta
import numpy as np
import scipy
import seaborn as sns
from tqdm.auto import tqdm  # jupyter compatible progress bar

tqdm.pandas()  # link tqdm to pandas
# Notebook magic
# save plots in the notebook
%matplotlib inline
# reloads modules automatically before executing cells
%load_ext autoreload
%autoreload 2
print(f"running in directory: {os.getcwd()}")  # where are we?
print(f"running on node: {socket.gethostname()}")  # what node are we on?

running in directory: /global/u1/p/pleung/grad_reqs
running on node: nid003556


### Insert the root of the crispy_shifty repo

In [2]:
sys.path.insert(0, "/global/cfs/cdirs/m4129/projects/crispy_shifty")

### Fix the paths
Necessary because we are on perlmutter

In [3]:
from crispy_shifty.utils.io import fix_path_prefixes

pairs = os.path.join(os.getcwd(), "01_mpnn_around_ligand/filtered_designed.pair")
new_pairs = fix_path_prefixes(
    find="/mnt/home/pleung/projects",
    replace="/global/u1/p/pleung",
    file=pairs,
    overwrite=True,
)

/global/cfs/cdirs/m4129/projects/crispy_shifty/envs/crispy/lib/python3.8/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


### Run AF2 on the MPNN designed binders

In [ ]:
from crispy_shifty.utils.io import gen_array_tasks

simulation_name = "02_fold_apo"
design_list_file = os.path.join(
    os.getcwd(), "01_mpnn_around_ligand/filtered_designed.pair"
)

output_path = os.path.join(f"/pscratch/sd/p/pleung/{simulation_name}")

options = " ".join(
    [
        "out:level 200",
        f"extra_res_fa {os.path.join(os.getcwd(), 'afo.params')}",
    ]
)
extra_kwargs = {"models": "4"}

gen_array_tasks(
    distribute_func="deployables.fold_binder",
    func_root="/global/u1/p/pleung/grad_reqs",
    design_list_file=design_list_file,
    output_path=output_path,
    perlmutter_mode=True,
    nstruct=1,
    nstruct_per_task=1,
    options=options,
    extra_kwargs=extra_kwargs,
    simulation_name=simulation_name,
    time="24:00",
)

### Set working directory to the root of the crispy_shifty repo
Note: We rsync the perlmutter results, now back on the digs

In [ ]:
os.chdir("/projects/crispy_shifty")

### Collect scorefiles of the folded binders and concatenate

In [ ]:
sys.path.insert(0, "/projects/crispy_shifty")
from crispy_shifty.utils.io import collect_score_file

simulation_name = "01_fold_binders"
output_path = os.path.join(os.getcwd(), f"projects/crispy_crispies/{simulation_name}")

if not os.path.exists(os.path.join(output_path, "scores.json")):
    collect_score_file(output_path, "scores")

### Load resulting concatenated scorefile

In [ ]:
sys.path.insert(0, "/projects/crispy_shifty")
from crispy_shifty.utils.io import parse_scorefile_linear

output_path = os.path.join(os.getcwd(), f"projects/crispy_crispies/{simulation_name}")

if not os.path.exists(os.path.join(output_path, "scores.csv")):
    scores_df = parse_scorefile_linear(os.path.join(output_path, "scores.json"))

### Dump scores_df as a CSV and then reload, for performance reasons

In [ ]:
if not os.path.exists(os.path.join(output_path, "scores.csv")):
    scores_df.to_csv(os.path.join(output_path, "scores.csv"))

scores_df = pd.read_csv(os.path.join(output_path, "scores.csv"), index_col="Unnamed: 0")

### Setup for plotting

In [ ]:
sns.set(
    context="talk",
    font_scale=1,  # make the font larger; default is pretty small
    style="ticks",  # make the background white with black lines
    palette="colorblind",  # a color palette that is colorblind friendly!
)

### Data exploration
Gonna remove the Rosetta sfxn scoreterms for now

In [ ]:
rosetta = scores_df[scores_df["designed_by"] == "rosetta"]
mpnn = scores_df[scores_df["designed_by"] == "mpnn"].sample(1000, random_state=0)
sample_df = pd.concat([rosetta, mpnn])

In [ ]:
from crispy_shifty.protocols.design import beta_nov16_terms

sample_df = sample_df[
    [term for term in sample_df.columns if term not in beta_nov16_terms]
]
print(len(sample_df))
print(list(sample_df.columns))

In [ ]:
from crispy_shifty.utils.plotting import histplot_df, pairplot_df

### Plot before and after interface design AF2 metrics

In [ ]:
cols = [
    "mean_pae_interaction",
    "mean_plddt",
    "pTMscore",
    "rmsd_to_reference",
]
the_fig = histplot_df(
    df=sample_df,
    cols=cols,
    bins=10,
    hue="designed_by",
)
plt.savefig(os.path.join(output_path, "old_vs_new_af2_scores.png"))

In [ ]:
mpnn["mpnn_temperature"] = mpnn["mpnn_temperature"].astype(str)

cols = [
    "best_average_plddts",
    "best_ptm",
    "best_rmsd_to_input",
    "mean_plddt",
    "pTMscore",
    "rmsd_to_reference",
]
the_fig = histplot_df(
    df=mpnn,
    cols=cols,
    bins=10,
    hue="mpnn_temperature",
)
plt.savefig(os.path.join(output_path, "old_vs_new_af2_scores_temp.png"))

In [ ]:
the_fig = pairplot_df(
    df=mpnn,
    cols=cols,
    hue="mpnn_temperature",
)
plt.savefig(os.path.join(output_path, "old_vs_new_af2_scores_temp_paired.png"))

In [ ]:
the_fig = histplot_df(
    df=mpnn,
    cols=cols,
    bins=10,
    hue="mpnn_design_area",
)
plt.savefig(os.path.join(output_path, "old_vs_new_af2_scores_area.png"))

In [ ]:
the_fig = pairplot_df(
    df=mpnn,
    cols=cols,
    hue="mpnn_design_area",
)
plt.savefig(os.path.join(output_path, "old_vs_new_af2_scores_area_paired.png"))

In [ ]:
# sample_df["best_model"] = sample_df["best_model"].astype(str)
# the_fig = histplot_df(
#     df=sample_df,
#     cols=cols,
#     bins=10,
#     hue="best_model",
# )

In [ ]:
# sample_df["best_model"] = sample_df["best_model"].astype(str)
# the_fig = histplot_df(
#     df=sample_df,
#     cols=cols,
#     bins=10,
#     hue="topo",
# )

### Plot interface metrics

In [ ]:
cols = [
    "cms_AnAcB",
    "mean_pae",
    "mean_pae_interaction",
    "mean_pae_intra_chain",
    "mean_plddt",
    "sc_AnAcB",
]
the_fig = histplot_df(
    df=sample_df,
    cols=cols,
    bins=10,
    hue="designed_by",
)
plt.savefig(os.path.join(output_path, "interface_scores.png"))

In [ ]:
the_fig = pairplot_df(
    df=sample_df,
    cols=cols,
    hue="designed_by",
)
plt.savefig(os.path.join(output_path, "interface_scores_paired.png"))

### Filter the whole df

In [ ]:
query = "mean_pae_interaction <10 and mean_plddt > 92 and rmsd_to_reference < 1.75"

filtered = scores_df.query(query)
len(filtered)

In [ ]:
filtered.groupby("target_name").count()

### Make seperate dfs of binders for the GIP/GLP/Glucagon family and for NPY

In [ ]:
gip = filtered.query("target_name == 'gip'")
glp = filtered.query("target_name == 'glp'")
gluc = filtered.query("target_name == 'glucagon'")
npy = filtered.query("target_name == 'neuropeptideY'")

### Save a list of outputs
Sort by length  

In [ ]:
simulation_name = "01_fold_binders"
output_path = os.path.join(os.getcwd(), f"projects/crispy_crispies/{simulation_name}")
filtered = filtered.sort_values("total_length")
with open(os.path.join(output_path, "folded_binders.list"), "w") as f:
    for path in tqdm(filtered.index):
        print(path, file=f)
with open(os.path.join(output_path, "gip.list"), "w") as f:
    for path in tqdm(gip.index):
        print(path, file=f)
with open(os.path.join(output_path, "glp.list"), "w") as f:
    for path in tqdm(glp.index):
        print(path, file=f)
with open(os.path.join(output_path, "gluc.list"), "w") as f:
    for path in tqdm(gluc.index):
        print(path, file=f)
with open(os.path.join(output_path, "npy.list"), "w") as f:
    for path in tqdm(npy.index):
        print(path, file=f)

### Fix the paths
Necessary because we transferred from perlmutter

In [ ]:
from crispy_shifty.utils.io import fix_path_prefixes

for l in "gip", "glp", "gluc", "npy":

    _ = fix_path_prefixes(
        find="/pscratch/sd/p/pleung",
        replace="/mnt/projects/crispy_shifty/projects/crispy_crispies",
        file=os.path.join(
            os.getcwd(), f"projects/crispy_crispies/01_fold_binders/{l}.list"
        ),
        overwrite=True,
    )

### Save a list of chunked outputs
50 per chunk

In [ ]:
from more_itertools import ichunked

with open(os.path.join(output_path, "folded_binders.pair"), "w") as f:
    for paths in ichunked(tqdm(filtered.index), 50):
        line = "____".join(paths)
        print(line, file=f)

### Prototyping blocks